# Widget Experiments for Creating UI to control Label Placement

In [ ]:
import asyncio
import json
import pathlib

import ipywidgets as W
import traitlets as T

import ipyelk
from ipyelk.diagram import layout_options

In [ ]:
nlp = layout_options.NodeLabelPlacement()
nsc = layout_options.NodeSizeConstraints()
nsm = layout_options.NodeSizeMinimum()

options_widgets = layout_options.OptionsWidget(options=[nlp, nsc, nsm])
options_widgets

### Elk Layout options are available off the `value` trait

In [ ]:
options_widgets.value

ELKJSON build acceptable value for the label placement

### Example connecting Layout Options widget value to the Simple Diagram

In [ ]:
simple = ipyelk.ElkDiagram()
sizer = ipyelk.diagram.ElkTextSizer()


async def update_label_layout_options(node, options):
    "Recurse through the nodes and update layout options"
    node.pop("width", None)
    node.pop("height", None)
    node["layoutOptions"] = options_widgets.value
    for label in node.get("labels", []):
        if "layoutOptions" in label:
            label["layoutOptions"].update(options)
        else:
            label["layoutOptions"] = options

        # removing rigid x and y placement
        label.pop("x", None)
        label.pop("y", None)

        # updating label size with measured values from the DOM
        size = await sizer.measure(label.get("text"))
        label["height"] = size.height
        label["width"] = size.width
    for child in node.get("children", []):
        await update_label_layout_options(child, options)


async def update_json():
    elk_json = json.loads(pathlib.Path("simple.json").read_text(encoding="utf-8"))
    elk_json["properties"] = {"algorithm": "layered"}
    layoutOptions = {nlp.identifier: nlp.value}
    await update_label_layout_options(elk_json, layoutOptions)
    simple.value = elk_json


def refresh(change=None):
    """Wrapper function for scheduling the async update_json function"""
    asyncio.create_task(update_json())


options_widgets.observe(refresh, "value")
refresh()
W.HBox([simple, options_widgets], layout={"height": "400px"})

In [ ]:
options_widgets.value

In [ ]:
from IPython.display import JSON

JSON(simple.value)